In [2]:
import re
from bs4 import BeautifulSoup
import datetime as dt
from urllib.request import urlopen
from db_utils import retrieve_data, generate_unique_game_id, get_rank_from_row
from scrape_all_games_data import get_daily_games_at_date
from io import StringIO
import pandas as pd

In [ ]:
# get_daily_games_at_date(dt.date(2019, 9, 20))

,date,home,home_points,visitor,visitor_points,home_rank,visitor_rank,unique_id
0,2019-09-20,Air Force,19,Boise State,30,None,20,8c4cf8c37a30e603e57f6a32f98121ea
0,2019-09-20,Florida International,31,Louisiana Tech,43,None,None,2d8031519c2508fba29981931f335a7f
0,2019-09-20,Utah,23,USC,30,10,None,a27f5a3f7bfd23b20eccd8c6b96d826c


In [4]:
# # date = 
# base_url = "https://www.sports-reference.com/cfb/boxscores/index.cgi"
# url = f"{base_url}/{date.strftime('%Y-%m-%d')}-{home}.html"

In [5]:
url = "https://www.sports-reference.com/cfb/boxscores/2025-01-20-notre-dame.html"
html = urlopen(url)
soup = BeautifulSoup(html, features="html.parser")
cand_divs = soup.find_all(attrs={"class": re.compile("linescore nohover")})

In [6]:
html_io = StringIO(str(cand_divs[0]))
df = pd.read_html(html_io)[0]

In [20]:
# double check OT formatting.
# TODO: get the unique ID to match it to
# visitor = 0
visitor_row = df.iloc[1]
visitor_row["Unnamed: 1"].replace(u'\xa0', ' ')
visitor = get_rank_from_row(visitor_row, "Unnamed: 1")[0]

In [18]:
visitor_row

Unnamed: 0    via Sports Logos.net About logos
Unnamed: 1                      Notre Dame (3)
1                                            7
2                                            0
3                                            8
4                                            8
Final                                       23
Name: 1, dtype: object

In [ ]:
# 1, 2, 3, 4 and if 5 

('Notre Dame', 3)

In [ ]:
["visitor", "home"]
["first_quarter", "first_half", "third_quarter", "fourth_quarter"]
for i in range(2):


In [24]:
get_rank_from_row([df["Unnamed: 1"][0].replace(u'\xa0', ' ')])

('Ohio State', 6)

In [25]:
visitor

'Notre Dame'

In [ ]:
import datetime as dt
# import pandas as pd
# import psycopg2
# import pytz
# import re
# import sys
# import time
# import warnings
# from bs4 import BeautifulSoup
# from io import StringIO
# from psycopg2.extras import execute_values
# from psycopg2.extensions import AsIs
# from typing import List
# from urllib.request import urlopen
# import sys
# # from src.db_utils import generate_unique_game_id, execute_sql_script
# from sports_betting.src.cfb.data.scrape_all_games_data import get_daily_games_at_date, clean_df, insert_daily_games_at_date, backfill_games
# from typing import Callable
# from db_utils import load_config, retrieve_data

In [1]:
import sys
print(sys.path)

['C:\\Users\\brtnl\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip', 'C:\\Users\\brtnl\\AppData\\Local\\Programs\\Python\\Python312\\DLLs', 'C:\\Users\\brtnl\\AppData\\Local\\Programs\\Python\\Python312\\Lib', 'C:\\Users\\brtnl\\AppData\\Local\\Programs\\Python\\Python312', 'c:\\Users\\brtnl\\OneDrive\\Desktop\\code\\sports_betting\\venv', '', 'c:\\Users\\brtnl\\OneDrive\\Desktop\\code\\sports_betting\\venv\\Lib\\site-packages', 'C:\\Users\\brtnl\\OneDrive\\Desktop\\code\\sports_betting\\src', 'c:\\Users\\brtnl\\OneDrive\\Desktop\\code\\sports_betting\\venv\\Lib\\site-packages\\win32', 'c:\\Users\\brtnl\\OneDrive\\Desktop\\code\\sports_betting\\venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\brtnl\\OneDrive\\Desktop\\code\\sports_betting\\venv\\Lib\\site-packages\\Pythonwin']


In [3]:
import db_utils

ModuleNotFoundError: No module named 'pandas'

In [2]:
from pip import _internal
_internal.main(['list'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Package           Version

----------------- -----------

asttokens         3.0.0

colorama          0.4.6

...

stack-data        0.6.3

tornado           6.4.2

traitlets         5.14.3

wcwidth           0.2.13

0

In [3]:
import sports_betting

ModuleNotFoundError: No module named 'sports_betting'

In [3]:

import sys
print(sys.path)

['C:\\Users\\brtnl\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip', 'C:\\Users\\brtnl\\AppData\\Local\\Programs\\Python\\Python312\\DLLs', 'C:\\Users\\brtnl\\AppData\\Local\\Programs\\Python\\Python312\\Lib', 'C:\\Users\\brtnl\\AppData\\Local\\Programs\\Python\\Python312', 'c:\\Users\\brtnl\\OneDrive\\Desktop\\code\\sports_betting\\venv', '', 'c:\\Users\\brtnl\\OneDrive\\Desktop\\code\\sports_betting\\venv\\Lib\\site-packages', 'c:\\Users\\brtnl\\OneDrive\\Desktop\\code\\sports_betting\\venv\\Lib\\site-packages\\win32', 'c:\\Users\\brtnl\\OneDrive\\Desktop\\code\\sports_betting\\venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\brtnl\\OneDrive\\Desktop\\code\\sports_betting\\venv\\Lib\\site-packages\\Pythonwin']


In [6]:
pip -V

pip 24.3.1 from c:\Users\brtnl\OneDrive\Desktop\code\sports_betting\venv\Lib\site-packages\pip (python 3.12)

Note: you may need to restart the kernel to use updated packages.


In [6]:
import sports_betting

ModuleNotFoundError: No module named 'sports_betting'

In [ ]:
def get_rank_from_row(row):
    pattern2 = r"^(.*?)\s\((\d{1,2})\)$"
    university = row[0]
    match = re.fullmatch(pattern2, university)
    if match:
         return match.group(1), int(match.group(2))
    else:
         return university, None

In [ ]:
get_rank_from_row(all_games_table[5].iloc[2])

('Arizona', '21')

In [ ]:
all_games_table[5]

,0,1,2
0,Sat 8/31,Sat 8/31,Sat 8/31
1,New Mexico,39,Final
2,Arizona (21),61,NaN


In [ ]:
temp.iloc[0][0]

'Western Kentucky'

In [ ]:
regex2 = r".*\(\b([1-9]|1[0-9]|2[0-5])\b\)"
re.fullmatch(regex2, "Western Kentucky (5)")

<re.Match object; span=(0, 20), match='Western Kentucky (5)'>

In [ ]:
temp = all_games_table[3][1:]
temp.loc[:, "rank"] = None
# temp

/var/folders/z_/5kxjclhd5090czxnlnxslydr0000gn/T/ipykernel_88211/1111760236.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.loc[:, "rank"] = None


regex = "^[A-Za-z]{3} \d{1,2}/\d{2}$"
string = all_games_table[3].iloc[0][0]

s = re.fullmatch(regex, string)

In [ ]:
s

<re.Match object; span=(0, 8), match='Sat 8/31'>

In [ ]:
month = month.lower()

    # URL to scrape, notice f string:
    url = (
        f"https://www.basketball-reference.com/leagues/NBA_%(year)s_games-%(month)s.html"
        % {"year": year, "month": month}
    )
    try:
        html = urlopen(url)
    except Exception as e:
        print(e)
        print(
            "Failure: %(month)s, %(year)s all_games URL not pulled"
            % {"month": month.capitalize(), "year": year}
        )
        return None

    # Kicks you out if you request over 20 times over a minute
    time.sleep(3.5)

    soup = BeautifulSoup(html, features="html.parser")

    # Convert the tables into a string and wrap it with StringIO
    tables = soup.find_all("table", {"id": re.compile("schedule")})
    html_string = "\n".join(str(table) for table in tables)
    html_io = StringIO(html_string)

    all_games_table = pd.read_html(html_io, header=0)[0]
    all_games_table = clean_all_games_table(all_games_table)
    if all_games_table is not None:
        all_games_table["Unique_ID"] = all_games_table.apply(
            generate_unique_game_id, axis=1
        )
    return all_games_table